In [1]:
import numpy as np
import pandas as pd

In [2]:
from thesis_tools.utils.data import *

In [3]:
billionaire_df = read_billionaires_data(year_and_month_int=True)
# drop region = 'Rest of World'
billionaire_df = billionaire_df[billionaire_df['region'] != 'Rest of World']
# drop sub_region = '"Not a sub-region"
billionaire_df = billionaire_df[billionaire_df['sub_region'] != 'Not a sub-region']

In [4]:
# What do we need to merge with?
# ISO codes
# Stock market data
# GDP data
# Population data -> calculate population weighted GDP per capita


In [5]:
iso_codes = read_iso_codes()

In [6]:
df = pd.merge(billionaire_df, iso_codes, left_on='country_of_citizenship', right_on='name', how='left')

In [7]:
# use year int and month int to get the year and month and make a date column that sets as date the last day of the month
df['date'] = pd.to_datetime(df['year_int'].astype(str) + '-' + df['month_int'].astype(str) + '-01') + pd.offsets.MonthEnd(0)

In [8]:
stock_market_data = read_stock_market_data()

In [9]:
# merge stock market data on dates
df = pd.merge(df, stock_market_data, left_on='date', right_on='Date', how='left')

In [10]:
df

,year_int,month_int,rank,net_worth,full_name,self_made,country_of_citizenship,region,sub_region,log_net_worth,ISO2,ISO3,date,Adj_Close_MSCI,Adj_Close_SPX
0,1997,1,NaN,2.0,Chatri Sophonpanich & family,False,Thailand,East Asia,Southeast Asia,0.693147,TH,THA,1997-01-31,822.299988,786.159973
1,1997,1,NaN,1.8,King Bhumibol Adulyadej,False,Thailand,East Asia,Southeast Asia,0.587787,TH,THA,1997-01-31,822.299988,786.159973
2,1999,1,NaN,1.0,Srichand & Gopichand Hinduja,False,India,India,India,0.000000,IN,IND,1999-01-31,1164.800049,1279.640015
3,1999,1,NaN,7.1,Dieter Schwarz,True,Germany,Europe,Germany,1.960095,DE,DEU,1999-01-31,1164.800049,1279.640015
4,1999,1,NaN,2.9,Martin Ebner,False,Switzerland,Europe,Alps,1.064711,CH,CHE,1999-01-31,1164.800049,1279.640015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28999,2023,1,2540.0,1.0,Yu Rong,True,China,China,China,0.000000,CN,CHN,2023-01-31,2820.699951,4076.600098
29000,2023,1,2540.0,1.0,"Richard Yuengling, Jr.",False,United States,North America,U.S.,0.000000,US,USA,2023-01-31,2820.699951,4076.600098
29001,2023,1,2540.0,1.0,Zhang Gongyun,True,China,China,China,0.000000,CN,CHN,2023-01-31,2820.699951,4076.600098
29002,2023,1,2540.0,1.0,Zhang Guiping & family,True,China,China,China,0.000000,CN,CHN,2023-01-31,2820.699951,4076.600098


In [11]:
population_data = read_population_data()
population_data.reset_index(inplace=True)
population_data = pd.merge(population_data, iso_codes, left_on='Name', right_on='name', how='left')
population_data.set_index('ISO3', inplace=True)
population_data.drop(columns=['Name', 'ISO2'], inplace=True)
population_data.columns = population_data.columns.astype(int)

In [12]:
# add population data
# for each row, find ISO 3 and year, and find the corresponding population
df['country_population'] = df.apply(lambda x: population_data.loc[x['ISO3'], x['year_int']], axis=1)

In [13]:
df

,year_int,month_int,rank,net_worth,full_name,self_made,country_of_citizenship,region,sub_region,log_net_worth,ISO2,ISO3,date,Adj_Close_MSCI,Adj_Close_SPX,country_population
0,1997,1,NaN,2.0,Chatri Sophonpanich & family,False,Thailand,East Asia,Southeast Asia,0.693147,TH,THA,1997-01-31,822.299988,786.159973,6.071530e+07
1,1997,1,NaN,1.8,King Bhumibol Adulyadej,False,Thailand,East Asia,Southeast Asia,0.587787,TH,THA,1997-01-31,822.299988,786.159973,6.071530e+07
2,1999,1,NaN,1.0,Srichand & Gopichand Hinduja,False,India,India,India,0.000000,IN,IND,1999-01-31,1164.800049,1279.640015,1.040716e+09
3,1999,1,NaN,7.1,Dieter Schwarz,True,Germany,Europe,Germany,1.960095,DE,DEU,1999-01-31,1164.800049,1279.640015,8.133380e+07
4,1999,1,NaN,2.9,Martin Ebner,False,Switzerland,Europe,Alps,1.064711,CH,CHE,1999-01-31,1164.800049,1279.640015,7.135000e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28999,2023,1,2540.0,1.0,Yu Rong,True,China,China,China,0.000000,CN,CHN,2023-01-31,2820.699951,4076.600098,1.425887e+09
29000,2023,1,2540.0,1.0,"Richard Yuengling, Jr.",False,United States,North America,U.S.,0.000000,US,USA,2023-01-31,2820.699951,4076.600098,3.382900e+08
29001,2023,1,2540.0,1.0,Zhang Gongyun,True,China,China,China,0.000000,CN,CHN,2023-01-31,2820.699951,4076.600098,1.425887e+09
29002,2023,1,2540.0,1.0,Zhang Guiping & family,True,China,China,China,0.000000,CN,CHN,2023-01-31,2820.699951,4076.600098,1.425887e+09


In [14]:
gdp_data = read_gdp_data()
gdp_data['TWN'] = 20000 # average over the interesting period
gdp_data['GGY'] = 30000 # average over the interesting period
gdp_data['PRK'] = gdp_data['KOR'] # North Korea is not in the data, so we use South Korea's GDP - there seems to be some confusion in the codes
# TODO: find data on Taiwan, Guernsey

In [16]:
# find gdp per capita based on year int and ISO3
df['gdp_per_capita'] = df.apply(lambda x: gdp_data.loc[str(x['year_int']), x['ISO3']], axis=1)

In [17]:
countries_by_region = {
    'North America': 
        ['United States', 'Canada'],
    'Europe': 
        ['Germany', 'United Kingdom', 'Ireland', 'Cyprus', 'Czech Republic', 'Czechia', 'Denmark', 'Austria',
        'Belgium', 'Spain', 'France', 'Greece', 'Italy', 'Netherlands', 'Norway', 'Poland', 'Portugal', 
        'Sweden', 'Switzerland', 'Liechtenstein', 'Lithuania', 'Monaco', 'Estonia', 'Finland', 'Slovakia', 
        'Romania', 'Hungary', 'Bulgaria', 'Guernsey', 'Iceland'],
    'China': 
        ['China', 'Hong Kong', 'Macau', 'Macao'],
    'East Asia': 
        ['Thailand', 'Malaysia', 'Singapore', 'Taiwan', 'Philippines', 'Indonesia', 'South Korea', 'Japan',
        'Australia', 'Vietnam', 'New Zealand'],
    'India': 
        ['India'],
    'Central Eurasia': 
        ['Russia', 'Kazakhstan', 'Ukraine', 'Armenia', 'Georgia'],
    'South America': 
        ['Brazil', 'Chile', 'Argentina', 'Peru', 'Venezuela', 'Colombia', 'Uruguay', 'Guatemala',
        'Panama', 'Barbados', 'Belize', 'Mexico'],
    'Middle East': 
        ['Turkey', 'Egypt', 'Israel', 'Saudi Arabia', 'United Arab Emirates', 'Kuwait', 'Qatar', 'Oman',
        'Lebanon'],
}

countries_by_sub_region = {
    'U.S.':
        ['United States'],
    'Canada':
        ['Canada'],
    'Germany':
        ['Germany'],
    'British Islands':
        ['United Kingdom', 'Ireland'],
    'Scandinavia':
        ['Denmark', 'Norway', 'Sweden', 'Finland'],
    'France':
        ['France', 'Monaco'],
    'Alps':
        ['Switzerland', 'Liechtenstein', 'Austria'],
    'Italy':
        ['Italy'],
    'China': 
        ['China', 'Hong Kong'],
    'Southeast Asia':
        ['Thailand', 'Malaysia', 'Singapore'],
    'Asian Islands':
        ['Taiwan', 'Philippines', 'Indonesia'],
    'South Korea':
        ['South Korea'],
    'Japan':
        ['Japan'],
    'Australia':
        ['Australia'],
    'India': 
        ['India'],
    'Russia':
        ['Russia'],
    'Brazil':
        ['Brazil'],
    'Israel + Turkey':
        ['Israel', 'Turkey']
}

In [18]:
# ISO_3 codes by region
ISO_3_by_region = {}
for region, countries in countries_by_region.items():
    # the country name is in the iso_codes dataframe index
    ISO_3_by_region[region] = iso_codes.loc[countries, 'ISO3'].values

# ISO_3 codes by sub region
ISO_3_by_sub_region = {}
for sub_region, countries in countries_by_sub_region.items():
    # the country name is in the iso_codes dataframe index
    ISO_3_by_sub_region[sub_region] = iso_codes.loc[countries, 'ISO3'].values

In [19]:
# make a population by region dataframe
population_by_region = {}
for region, ISO_3s in ISO_3_by_region.items():
    population_by_year = {}
    for year in population_data.columns:
        population_by_year[year] = population_data.loc[ISO_3s, year].sum()
    population_by_region[region] = population_by_year
population_by_region = pd.DataFrame(population_by_region)

# make a population by sub region dataframe
population_by_sub_region = {}
for sub_region, ISO_3s in ISO_3_by_sub_region.items():
    population_by_year = {}
    for year in population_data.columns:
        population_by_year[year] = population_data.loc[ISO_3s, year].sum()
    population_by_sub_region[sub_region] = population_by_year
population_by_sub_region = pd.DataFrame(population_by_sub_region)

In [20]:
# make a weighted gdp per capita by region dataframe
gdp_per_capita_by_region = {}
for region, ISO_3s in ISO_3_by_region.items():
    gdp_per_capita_by_year = {}
    for year in population_by_region.index:
        year = int(year)
        divisor = population_by_region.loc[year, region]
        sum = (gdp_data.loc[str(year), ISO_3s] * population_data.loc[ISO_3s, year]).sum()
        gdp_per_capita_by_year[year] = sum / divisor 
    gdp_per_capita_by_region[region] = gdp_per_capita_by_year
gdp_per_capita_by_region = pd.DataFrame(gdp_per_capita_by_region)

# make a weighted gdp per capita by sub region dataframe
gdp_per_capita_by_sub_region = {}
for sub_region, ISO_3s in ISO_3_by_sub_region.items():
    gdp_per_capita_by_year = {}
    for year in population_by_sub_region.index:
        year = int(year)
        divisor = population_by_sub_region.loc[year, sub_region]
        sum = (gdp_data.loc[str(year), ISO_3s] * population_data.loc[ISO_3s, year]).sum()
        gdp_per_capita_by_year[year] = sum / divisor 
    gdp_per_capita_by_sub_region[sub_region] = gdp_per_capita_by_year
gdp_per_capita_by_sub_region = pd.DataFrame(gdp_per_capita_by_sub_region)

In [24]:
df['region_gdp_per_capita'] = df.apply(lambda x: gdp_per_capita_by_region.loc[x['year_int'], x['region']], axis=1)
df['sub_region_gdp_per_capita'] = df.apply(lambda x: gdp_per_capita_by_sub_region.loc[x['year_int'], x['sub_region']], axis=1)

In [85]:
import copy
panel_df = copy.deepcopy(df)

In [86]:
panel_df = panel_df[['year_int', 'country_of_citizenship', 'region', 'sub_region', 'gdp_per_capita', 'region_gdp_per_capita', 'sub_region_gdp_per_capita', 'Adj_Close_MSCI', 'Adj_Close_SPX', 'net_worth']]

In [87]:
panel_df

,year_int,country_of_citizenship,region,sub_region,gdp_per_capita,region_gdp_per_capita,sub_region_gdp_per_capita,Adj_Close_MSCI,Adj_Close_SPX,net_worth
0,1997,Thailand,East Asia,Southeast Asia,2462.414380,10201.978863,4063.950655,822.299988,786.159973,2.0
1,1997,Thailand,East Asia,Southeast Asia,2462.414380,10201.978863,4063.950655,822.299988,786.159973,1.8
2,1999,India,India,India,440.961455,440.961455,440.961455,1164.800049,1279.640015,1.0
3,1999,Germany,Europe,Germany,26734.942537,20099.290430,26734.942537,1164.800049,1279.640015,7.1
4,1999,Switzerland,Europe,Alps,41695.691291,20099.290430,34137.905321,1164.800049,1279.640015,2.9
...,...,...,...,...,...,...,...,...,...,...
28999,2023,China,China,China,12720.216318,13018.586592,12909.682755,2820.699951,4076.600098,1.0
29000,2023,United States,North America,U.S.,76329.582265,74205.812199,76329.582265,2820.699951,4076.600098,1.0
29001,2023,China,China,China,12720.216318,13018.586592,12909.682755,2820.699951,4076.600098,1.0
29002,2023,China,China,China,12720.216318,13018.586592,12909.682755,2820.699951,4076.600098,1.0


In [88]:
AGGREGATE_TYPE = 'region'
# AGGREGATE_TYPE = 'region'
# AGGREGATE_TYPE = 'country_of_citizenship'
data = {}
if AGGREGATE_TYPE == 'sub_region':
    for sub_region in panel_df['sub_region'].unique():
        for year in panel_df[panel_df['sub_region']==sub_region]['year_int'].unique():
            data[(sub_region, year)] = panel_df[(panel_df['sub_region'] == sub_region) & (panel_df['year_int'] == year)]['net_worth']
    panel_df = panel_df[['sub_region', 'year_int', 'sub_region_gdp_per_capita', 'Adj_Close_MSCI', 'Adj_Close_SPX']]
elif AGGREGATE_TYPE == 'region':
    for region in panel_df['region'].unique():
        for year in panel_df[panel_df['region']==region]['year_int'].unique():
            data[(region, year)] = panel_df[(panel_df['region'] == region) & (panel_df['year_int'] == year)]['net_worth']
    panel_df = panel_df[['region', 'year_int', 'region_gdp_per_capita', 'Adj_Close_MSCI', 'Adj_Close_SPX']]
elif AGGREGATE_TYPE == 'country_of_citizenship':
    for country in panel_df['country_of_citizenship'].unique():
        for year in panel_df[panel_df['country_of_citizenship']==country]['year_int'].unique():
            data[(country, year)] = panel_df[(panel_df['country_of_citizenship'] == country) & (panel_df['year_int'] == year)]['net_worth']
    panel_df = panel_df[['country_of_citizenship', 'year_int', 'gdp_per_capita', 'Adj_Close_MSCI', 'Adj_Close_SPX']]

In [89]:
panel_df = panel_df.drop_duplicates()

In [90]:
panel_df['net_worth'] = panel_df.apply(lambda x: data[(x[AGGREGATE_TYPE], x['year_int'])].tolist(), axis=1)

In [91]:
panel_df['len_net_worth'] = panel_df['net_worth'].apply(len)

In [92]:
panel_df

,region,year_int,region_gdp_per_capita,Adj_Close_MSCI,Adj_Close_SPX,net_worth,len_net_worth
0,East Asia,1997,10201.978863,822.299988,786.159973,"[2.0, 1.8]",2
2,India,1999,440.961455,1164.800049,1279.640015,[1.0],1
3,Europe,1999,20099.290430,1164.800049,1279.640015,"[7.1, 2.9, 1.8]",3
5,South America,1999,4393.185130,1164.800049,1279.640015,[1.0],1
6,China,1999,1041.003028,1164.800049,1279.640015,[1.1],1
...,...,...,...,...,...,...,...
26568,China,2023,13018.586592,2820.699951,4076.600098,"[68.0, 45.0, 38.0, 35.3, 33.4, 30.2, 29.5, 26....",561
26592,East Asia,2023,14255.690727,2820.699951,4076.600098,"[32.6, 27.0, 25.5, 24.2, 23.1, 22.4, 21.0, 19....",293
26610,Central Eurasia,2023,12633.541610,2820.699951,4076.600098,"[25.2, 23.7, 22.1, 21.6, 20.9, 20.5, 18.5, 14....",104
26637,Middle East,2023,15093.452091,2820.699951,4076.600098,"[18.9, 14.0, 7.0, 7.0, 6.8, 6.3, 5.6, 5.3, 5.0...",56
